## Obtain the Annulus Domain

In [ ]:
import  numpy as np

from    sympde.topology            import Square, PolarMapping
from    sympde.utilities.utils     import plot_domain

# rmin and rmax define the inner and outer radius of the annulus
rmin, rmax      = 0.3, 1.

logical_domain  = Square('S', bounds1=(0., 1.), bounds2=(0., 2*np.pi))
F               = PolarMapping('A', dim=2, c1=0., c2=0., rmin=rmin, rmax=rmax)
domain          = F(logical_domain)

# periodicity of the domain
periodic        = [False, True] 

plot_domain(domain, draw=True, isolines=True)

## Discrete Model using de Rham objects

### Derham and Mass Matrices First

In [ ]:
from sympde.calculus            import dot
from sympde.expr                import LinearForm, BilinearForm, integral
from sympde.topology            import elements_of, Derham

derham = Derham(domain, sequence=['h1', 'hcurl', 'l2'])

V0      = derham.V0
V1      = derham.V1
V2      = derham.V2

u0, v0  = elements_of(V0, names='u0, v0')
u1, v1  = elements_of(V1, names='u1, v1')
u2, v2  = elements_of(V2, names='u2, v2')

# bilinear forms corresponding to V0, V1 and V2 mass matrices
m0      = BilinearForm((u0, v0), integral(domain, u0*v0))
m1      = BilinearForm((u1, v1), integral(domain, dot(u1, v1)))
m2      = BilinearForm((u2, v2), integral(domain, u2*v2))

In [ ]:
from psydac.api.discretization  import discretize
from psydac.api.settings        import PSYDAC_BACKEND_GPYCCEL

backend = PSYDAC_BACKEND_GPYCCEL

In [ ]:
ncells  = [32, 32]      # Bspline cells
degree  = [4, 4]        # Bspline degree

In [ ]:
# discretize domain and derham
domain_h    = discretize(domain, ncells=ncells, periodic=periodic)
derham_h    = discretize(derham, domain_h, degree=degree)

# define FEM spaces V0_h, V1_h and V2_h
V0h         = derham_h.V0
V1h         = derham_h.V1
V2h         = derham_h.V2

# Commuting projection operators
P0, P1, P2  = derham_h.projectors()

# Exterior derivative operators (grad and curl)
G, C        = derham_h.derivatives_as_matrices
Gt          = G.T
Ct          = C.T

# Mass matrices
m0_h        = discretize(m0, domain_h, (V0h, V0h), backend=backend)
m1_h        = discretize(m1, domain_h, (V1h, V1h), backend=backend)
m2_h        = discretize(m2, domain_h, (V2h, V2h), backend=backend)

M0          = m0_h.assemble()
M1          = m1_h.assemble()
M2          = m2_h.assemble()

## Boundary Conditions

We choose to apply the projection method. For that matter, we use the projection matrices
$$
\begin{align*}
    &\mathbb{P}_{H_0^1}, \text{ projecting into the coefficients space of functions in }H_0^1 \\
    &\mathbb{P}_{H_0(curl)}, \text{ projecting into the coefficients space of functions in }H_0(curl)
\end{align*}
$$
as well as the projection matrices
$$
\begin{align*}
    &\mathbb{P}_{H_0^1}^{\Gamma} = \mathbb{I}_0 - \mathbb{P}_{H_0^1} \\
    &\mathbb{P}_{H_0(curl)}^{\Gamma} = \mathbb{I}_1 - \mathbb{P}_{H_0(curl)}
\end{align*}
$$
The latter two matrices are used for stabilization purposes, i.e., in order to keep the system matrix non-singular.

In [ ]:
from psydac.linalg.basic    import IdentityOperator

from utils                  import H1BoundaryProjector2D, HcurlBoundaryProjector2D

P_H1            = H1BoundaryProjector2D(V0, V0h.coeff_space, periodic=periodic)
P_Hcurl         = HcurlBoundaryProjector2D(V1, V1h.coeff_space, periodic=periodic)

I0              = IdentityOperator(V0h.coeff_space)
I1              = IdentityOperator(V1h.coeff_space)

P_H1_Gamma      = I0 - P_H1
P_Hcurl_Gamma   = I1 - P_Hcurl

## Boundary Conditions and inverse Matrices

Let us denote the interior coefficients by a subscript $I$ and the exterior coefficients by a subscript $\Gamma$. Suppose we want to solve the linear system
$$
\begin{align*}
    \mathbb{M}_{I,I} x_{I} = b_{I}
\end{align*}
$$
but we only have access to the larger matrices and vectors
$$
\begin{align*}
    &\mathbb{M} = \left(\begin{matrix} \mathbb{M}_{I,I} && \mathbb{M}_{I,\Gamma} \\
                                      \mathbb{M}_{\Gamma,I} && \mathbb{M}_{\Gamma,\Gamma}
                       \end{matrix}\right), \\
    &b = \left(\begin{matrix} b_{I} \\
                               b_{\Gamma}
                \end{matrix}\right).
\end{align*}
$$
One way to do this is to solve the following system
$$
\begin{align*}
    \left(\begin{matrix} \mathbb{M}_{I,I} && \mathbb{0} \\
                                      \mathbb{0} && \mathbb{I}_{\Gamma,\Gamma}
          \end{matrix}\right)
    \left(\begin{matrix} x_{I} \\ x_{\Gamma} \end{matrix}\right) = 
    \left(\begin{matrix} b_{I} \\ 0 \end{matrix}\right)
\end{align*}
$$

In [ ]:
from psydac.linalg.solvers import inverse

M0_0    = P_H1 @ M0 @ P_H1 + P_H1_Gamma
M0_inv  = inverse(M0_0, 'cg', maxiter=1000, tol=1e-9)

# Build the discrete $\mathcal{L}_h^1$ operator in sparse format

Psydac does not offer a direct way to access eigenvalues and -vectors of matrices.
However, we can transform our operators to scipy.sparse format and use scipy's `eigsh` method.

We want our solution vector field $A$ to be an element of $(\mathfrak{h}_h^1)^{\perp}$, 
and it holds $\ker\mathcal{L}_h^1 = \mathfrak{h}_h^1$.

Because the annulus has one whole, and hence the space of harmonic forms is of dimension 1, we want to compute the first eigenvector (corresponding to the eigenvalue $0$) of the operator
$$
\mathcal{L}_h^1 = \widetilde{\boldsymbol{curl}}_h\ curl - \boldsymbol{grad}\ \widetilde{div}_h
$$
in order to later enforce the orthogonality constraint.

In [ ]:
from scipy.sparse           import csc_matrix, bmat
from scipy.sparse.linalg    import inv

# Sparse representations of projection operators
P_H1_sp             = csc_matrix(P_H1.toarray())
P_Hcurl_sp          = csc_matrix(P_Hcurl.toarray())

I0_sp               = csc_matrix(I0.toarray())
I1_sp               = csc_matrix(I1.toarray())

P_H1_Gamma_sp       = I0_sp - P_H1_sp
P_Hcurl_Gamma_sp    = I1_sp - P_Hcurl_sp

# Sparse representations of mass matrices and differentiation operators
M0_sp               = M0.tosparse()
M1_sp               = M1.tosparse()
M2_sp               = M2.tosparse()

G_sp                = G.tosparse()
C_sp                = C.tosparse()

Gt_sp               = G_sp.transpose()
Ct_sp               = C_sp.transpose()

# It can be shown that for the projection method to work 
# we need a modified version of the gradient operator
G_sp_0              = G.tosparse() @ P_H1_sp
Gt_sp_0             = G_sp_0.transpose()

# ... and of course the correct inverse M0 mass matrix in its sparse representation
M0_sp_0             = P_H1_sp @ M0.tosparse() @ P_H1_sp + P_H1_Gamma_sp
inv_M0_sp           = inv(M0_sp_0.tocsc())
inv_M0_sp.eliminate_zeros()

# For this complex operator, the projection method consists of using the modified gradient
L1_sp               = Ct_sp @ M2_sp @ C_sp + M1_sp @ G_sp_0 @ inv_M0_sp @ Gt_sp_0 @ M1_sp
# ... and the projection matrices as usual
L1_sp_bc            = P_Hcurl_sp @ L1_sp @ P_Hcurl_sp + P_Hcurl_Gamma_sp

# Compute first eigenvector of $\mathcal{L}_h^1$

In [ ]:
from utils                  import get_eigenvalues

dim_harmonic_space          = 1
eigenvalues, eigenvectors   = get_eigenvalues(dim_harmonic_space, 1e-6, L1_sp_bc, M1_sp)

# A basis of the vector space of harmonic forms (hf)
hf                  = eigenvectors[:,0]
# Obtain the eigenvector as a "row csc_matrix (1 x n)"
hft_sp              = csc_matrix(hf)
# ... and as a "column csc_matrix (n x 1)"
hf_sp               = hft_sp.transpose()

# Build system matrix in sparse format
... and apply projection method to enforce boundary conditions

In [ ]:
# System matrix without taking BCs into account
A_sp = bmat([[M0_sp,            Gt_sp @ M1_sp,          None],
             [M1_sp @ G_sp,     Ct_sp @ M2_sp @ C_sp,   M1_sp @ hf_sp],
             [None,             hft_sp @ M1_sp,         None]])

ZERO    = csc_matrix(np.array([0]))
ONE     = csc_matrix(np.array([1]))

P       = bmat([[P_H1_sp, None,       None],
                [None,    P_Hcurl_sp, None],
                [None,    None,       ONE]])

P_Gamma = bmat([[P_H1_Gamma_sp, None,               None],
                [None,          P_Hcurl_Gamma_sp,   None],
                [None,          None,               ZERO]])

# System matrix taking BCs into account
A_sp_bc = P @ A_sp @ P + P_Gamma

# Method of manufactured solution

In order to test our code, we employ the method of manufactured solution.
We start from the true $\boldsymbol{A}\in H_0(curl;\Omega),\ div\ \boldsymbol{A} = 0$
$$
\boldsymbol{A}(x, y) = \frac{1}{(x^2 + y^2)^{3/2}}\left(\begin{matrix} xy \\ y^2 \end{matrix}\right)
$$
Because the r.h.s. of the discrete variational formulation reads
$$
r.h.s. = \left(\begin{matrix} \mathbb{0} \\ (\boldsymbol{v},\ \boldsymbol{J}) \\ \mathbb{0} \end{matrix}\right)
$$
and because $\boldsymbol{J} = \boldsymbol{curl}\ curl\ \boldsymbol{A}$, the r.h.s. of the linear system becomes
$$
r.h.s. = \left(\begin{matrix} \mathbb{0} \\ \mathbb{C}^T\mathbb{M}_2\mathbb{C}\mathbb{A} \\ \mathbb{0} \end{matrix}\right)
$$
with $\mathbb{A}$ being the coefficient vector of discrete $\boldsymbol{A}$. If our code is correct, we will obtain approximately $\mathbb{A}$ as second component of the solution vector.

In [ ]:
# Define exact solution A_ex for method of manufactured solution
r       = lambda x, y : np.sqrt(x**2 + y**2)
A_ex_1  = lambda x, y : (x*y)  / (r(x,y)**3)
A_ex_2  = lambda x, y : (y**2) / (r(x,y)**3)

A_ex            = (A_ex_1, A_ex_2)
A_ex_FemField   = P1(A_ex)
A_ex_coeffs     = A_ex_FemField.coeffs

# First component of rhs
rhs1_nparray    = np.zeros(V0h.nbasis)

# Second component of rhs
rhs2_nparray    = (P_Hcurl @ Ct @ M2 @ C @ A_ex_coeffs ).toarray()

# Third component of rhs
rhs3_nparray    = np.zeros(dim_harmonic_space)

# Full rhs
rhs_nparray     = np.block([rhs1_nparray, rhs2_nparray, rhs3_nparray])

# -------------- direct solve with scipy spsolve ---------------
import  time
from    scipy.sparse.linalg import spsolve
t0 = time.time()
sol_nparray = spsolve(A_sp_bc.asformat('csr'), rhs_nparray)
t1 = time.time()
# --------------------------------------------------------------

## Error Analysis

In [ ]:
sigmah_nparray  = sol_nparray[:V0h.nbasis]
Ah_nparray      = sol_nparray[V0h.nbasis:V0h.nbasis + V1h.nbasis]
ph_nparray      = sol_nparray[V0h.nbasis+V1h.nbasis:]

from psydac.linalg.utilities import array_to_psydac
Ah_coeffs       = array_to_psydac(Ah_nparray, V1h.coeff_space)

error           = A_ex_coeffs - Ah_coeffs
l2_error        = np.sqrt( error.dot(M1 @ error) )

# print the result
print( '> Grid          :: [{ne1},{ne2}]'.format( ne1=ncells[0], ne2=ncells[1]) )
print( '> Degree        :: [{p1},{p2}]'  .format( p1=degree[0], p2=degree[1] ) )
print( '> L2 error      :: {:.2e}'.format( l2_error ) )
print( '' )
print( '> Solution time :: {:.3g}'.format( t1-t0 ) )